In [49]:
import requests 
from bs4 import BeautifulSoup 
import re 
import datetime as dt
from datetime import datetime, date, time
import pandas as pd

class extracter(object):
    
    # класс хранит ссылки на статьи, сами статьи с заголовками и даты загрузки статей
    def __init__(self):
        self.dates = []
        self.links = []
        self.titles = []
        self.articles = []
        
    # метод, выгружающий статьи, обязательные аргументы которой - адрес страницы и точка отсчета статей
    def extract_articles(self, url, startingpoint, endpoint=dt.datetime.today().date()):
        date = endpoint
        element = 0
        pagenum = 2
        
        '''На сайте нельзя найти статьи за определенную дату, т.к. нужно знать заранее название статьи,
        поэтому нужно пробегать по каждой дате на странице и переходить на следующую, если они закончились,
        а нужно еще, отсюда изначально pagenum = 2, на случай если придется пойти на вторую страницу, 
        в результате чего номер поменяетя на тройку и т.д.'''
        
        while date >= startingpoint:
            while self.dates_converter(url)[element] >= date:
                self.extract_article(self.extract_links(url)[element])
                self.dates.append(self.extract_dates(url)[element])
                self.links.append(self.extract_links(url)[element])
                print('{0} is done'.format(self.dates_converter(url)[element]))
                if element == len(self.dates_converter(url))-1:
                    url = url+"page/"+str(pagenum)+"/"
                    pagenum+=1
                    element = 0
                element+=1
            date = date - dt.timedelta(days=1)      
                
    # метод, приводящий даты на странице в нужный формат           
    def dates_converter(self, url):
        dates=[]
        for date in self.extract_dates(url):
            date = datetime.strptime(date, "%B %d, %Y").date()
            dates.append(date)
        return (dates)
    
    # метод, извлекающий даты со страницы    
    def extract_dates(self, url):
        dates = []
        page = requests.get(url)
        bs = BeautifulSoup(page.text, "html5lib")
        h2s=bs.find_all("h2", {"class": "date-header"})
        for e in h2s:
            dates.append(e.text)
        return (dates)
    
    # метод извлекает статью и ее название и кладет их в список
    def extract_article(self, url):
        pagetext=requests.get(url)
        bs=BeautifulSoup(pagetext.text, "html5lib")
        title = bs.find_all("h3", {"class": "entry-header"})
        article = bs.findAll('div', {"class": "entry-body"})
        
        replacements = {'&raquo;' : '"', '&laquo;' : '"', '\xa0' : ' ', 
                       '\n' : '', '\t' : '', '&nbsp;' : '', '&mdash;' : ' -'}
        
        def replacement(text, reps):
            for key, val in reps.items():
                text = text.replace(key, val)
            return(text)
        
        self.titles.append(replacement(title[0].text, replacements))
        self.articles.append(replacement(article[0].text, replacements))
        

    # метод извлекает со страницы все ссылки на статьи в список 
    # (каждому эллементу списка ссылок должен соответсвовать эелемент списка дат) 
    def extract_links(self, url):
        page = requests.get(url)
        bs=BeautifulSoup(page.text, "html5lib")
        h3s=bs.findAll('h3', {"class": "entry-header"})
        links=[]
        for e in range(len(h3s)):
            notalinkyet = h3s[e].findAll('a')
            almostlink=re.compile("<a href=(.+)>.+?", re.S).findall(str(notalinkyet[0]))
            links.append(almostlink[0][1:-1])
        return (links)
    
    # метод загрузки данных в csv файл
    def insert_to_csv(self, path):
        df = pd.DataFrame({'date':[], 'url':[], 'title':[], 'article':[]})
        for e in range(len(self.dates)):
            df = df.append({'date': self.dates[e], 
                            'url':self.links[e], 
                            'title':self.titles[e], 
                            'article':self.articles[e]}, ignore_index=True)
        df.to_csv(path+'\out.csv', sep = ';')
        return (df)
        

In [50]:
load = extracter()
url = 'http://blog.revolutionanalytics.com/'
load.extract_articles(url, startingpoint = dt.datetime.today().date() - dt.timedelta(days=30))

2018-10-12 is done
2018-10-11 is done
2018-10-09 is done
2018-10-05 is done
2018-10-03 is done
2018-10-02 is done
2018-09-28 is done
2018-09-26 is done
2018-09-21 is done
2018-09-20 is done
2018-09-18 is done


In [51]:
load.insert_to_csv("D:\prog\homeworks")

,article,date,title,url
0,It seemed like such a simple movie. The Naked ...,"October 12, 2018","Because it's Friday: Hey, it's Enrico Pallazzo!",http://blog.revolutionanalytics.com/2018/10/be...
1,The Economist's Big Mac Index (also described ...,"October 11, 2018",The Economist's Big Mac Index is calculated wi...,http://blog.revolutionanalytics.com/2018/10/bi...
2,I wasn't at the Use of R in Official Statistic...,"October 09, 2018",How R gets built on Windows,http://blog.revolutionanalytics.com/2018/10/ho...
3,"Since 2015, the R Consortium has funded projec...","October 05, 2018",R Consortium grant applications due October 31,http://blog.revolutionanalytics.com/2018/10/r-...
4,Thanks to Mike Loukides I recently discovered ...,"October 03, 2018",Because it's Friday: If IKEA did algorithms,http://blog.revolutionanalytics.com/2018/10/be...
5,Here are some conferences focused on R taking ...,"October 02, 2018",A few upcoming R conferences,http://blog.revolutionanalytics.com/2018/10/a-...
6,"In case you missed them, here are some article...","September 28, 2018",In case you missed it: September 2018 roundup,http://blog.revolutionanalytics.com/2018/10/in...
7,"Microsoft Ignite, Microsoft's annual developer...","September 26, 2018","AI, Machine Learning and Data Science Announce...",http://blog.revolutionanalytics.com/2018/10/ig...
8,I was about the same age as student pilot Magg...,"September 21, 2018",Because it's Friday: Fly Strong,http://blog.revolutionanalytics.com/2018/09/be...
9,During the EARL (Enterprise Applications of th...,"September 20, 2018",Applications of R presented at EARL London 2018,http://blog.revolutionanalytics.com/2018/09/ea...


In [52]:
out = pd.read_csv('D:\prog\homeworks\out.csv', sep=';', index_col=0)
print(out.head())

                                             article              date  \
0  It seemed like such a simple movie. The Naked ...  October 12, 2018   
1  The Economist's Big Mac Index (also described ...  October 11, 2018   
2  I wasn't at the Use of R in Official Statistic...  October 09, 2018   
3  Since 2015, the R Consortium has funded projec...  October 05, 2018   
4  Thanks to Mike Loukides I recently discovered ...  October 03, 2018   

                                               title  \
0    Because it's Friday: Hey, it's Enrico Pallazzo!   
1  The Economist's Big Mac Index is calculated wi...   
2                        How R gets built on Windows   
3     R Consortium grant applications due October 31   
4        Because it's Friday: If IKEA did algorithms   

                                                 url  
0  http://blog.revolutionanalytics.com/2018/10/be...  
1  http://blog.revolutionanalytics.com/2018/10/bi...  
2  http://blog.revolutionanalytics.com/2018/10/ho... 